# **1. Write a program to demonstate the working of the Decision Tree based ID3 algorithm**

# **Decision Tree - ID3 Algorithm**
### (*ID3 - Entropy, Information Gain*)

- **ID3(*Examples*, TargetAttribute, Attribute)**


---


1.   Create a Root node for the tree.
2.   If all *Examples* are Positive, Return the single-node tree Root, with label = '+'.
3.   If all *Examples* are Negative, Return the single-node tree Root, with label = '-'.
4.   If Atributes is empty, Return the single-node tree Root, with label = Most Common value of TargetAttribute in *Examples*.
5.   Otherwise:
     - A <- the attribute from Attributes that best classifies *Examples*.
     - The decision attribute for Root <- A
     - For each possible value, vi, of A,
        - Add a new tree branch below Root, corresponding to the test A = vi.
        - Let *Examples*(vi) be the subset of *Examples* that have value vi for A.
        - If *Examples*(vi) is empty
            - Then below this new branch add a leaf node with label = Most common value of TargetAttribute in *Examples*.
            - Else below this new branch add the subtree ID3(*Examples*, TargetAttribute, Attributes-{A})
6.    End
7.    Root


---








In [3]:
import pandas as pd
from pandas import DataFrame
import math
from collections import Counter

In [4]:
df = pd.read_csv('/content/PlayTennis.csv')
df

,Unnamed: 0,PlayTennis,Outlook,Temperature,Humidity,Wind
0,0,No,Sunny,Hot,High,Weak
1,1,No,Sunny,Hot,High,Strong
2,2,Yes,Overcast,Hot,High,Weak
3,3,Yes,Rain,Mild,High,Weak
4,4,Yes,Rain,Cool,Normal,Weak
5,5,No,Rain,Cool,Normal,Strong
6,6,Yes,Overcast,Cool,Normal,Strong
7,7,No,Sunny,Mild,High,Weak
8,8,Yes,Sunny,Cool,Normal,Weak
9,9,Yes,Rain,Mild,Normal,Weak


In [20]:
df = df.drop(columns = ['Unnamed: 0'])

### **Step 1 - Calculating Entropy of Whole DataSet**





In [21]:
def entropy(probs):
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

def entropy_of_list(a_list):  
    from collections import Counter
    cnt = Counter(x for x in a_list)  
    num_instances = len(a_list)*1.0   
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]  

    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))

    return entropy(probs) 
    
# The initial entropy of the YES/NO attribute for our dataset.
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df['PlayTennis'])
total_entropy = entropy_of_list(df['PlayTennis'])
print("\n Total Entropy of PlayTennis Data Set:",total_entropy)


  INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: PlayTennis, dtype: object

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:

 Total Entropy of PlayTennis Data Set: 0.9402859586706309


### **Step 2 - Information Gain of Attributes**





In [37]:
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("==============================================================")
    print("INFORMATION GAIN CALCULATION OF OUTLOOK - ",split_attribute_name)
    '''
    Takes a DataFrame of attributes, and quantifies the entropy of a target
    attribute after performing a split along the values of another attribute.
    '''
    df_split = df.groupby(split_attribute_name)
   
    nobs = len(df.index) * 1.0
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    
    
    # Calculate Information Gain:
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


### **Step 2 - ID3 Algorithm**


In [38]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    
    if len(cnt) == 1:
        return next(iter(cnt))  
    elif df.empty or (not attribute_names):
        return default_class  


    else:
        default_class = max(cnt.keys()) 
        # Compute the Information Gain of the attributes:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) 
        # Choose Best Attribute to split on:
        best_attr = attribute_names[index_of_max]
        
        # Create an empty tree, to be populated in a moment
        tree = {best_attr:{}} 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
        # Split dataset
        # On each split, recursively call this algorithm.
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [39]:
attribute_names = list(df.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('PlayTennis') #Remove the class attribute 
print("Predicting Attributes:", attribute_names)

from pprint import pprint
tree = id3(df,'PlayTennis',attribute_names)

List of Attributes: ['PlayTennis', 'Outlook', 'Temperature', 'Humidity', 'Wind']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']
INFORMATION GAIN CALCULATION OF OUTLOOK -  Outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:
INFORMATION GAIN CALCULATION OF OUTLOOK -  Temperature

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.25:
 
 Pro

In [43]:
print("\nTHE RESULTANT DECISION TREE IS  :\n")
pprint(tree)
attribute = next(iter(tree))
print("BEST ATTRIBUTE :\n",attribute,"\n")
print("TREE KEYS:\n",tree[attribute].keys())


THE RESULTANT DECISION TREE IS  :

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
BEST ATTRIBUTE :
 Outlook 

TREE KEYS:
 dict_keys(['Overcast', 'Rain', 'Sunny'])
